In [25]:
from requests import Session
import pandas as pd
import time
import hashlib
import hmac
import base64
import json

# Import Classified

In [26]:
import sys
sys.path.append('..')
import classified

# GLOBAL

In [27]:
api_key = classified.API_KEY
api_secret = classified.API_SECRET
api_passphrase = classified.API_PASSPHRASE

In [28]:
def authentication(method, endpoint):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": "2"
    }
    return headers

# Private API

In [29]:
class PrivateData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        self.session = Session()

    # returns account balance
    def getBalance(self):
        url = self.apiurl + '/api/v1/accounts'
        headers = authentication('get', '/api/v1/accounts')
        r = self.session.get(url, headers=headers)
        data = r.json()["data"]
        df = pd.DataFrame.from_dict(data)
        return df
    
    # returns the full order book for a trading pair
    def getFullOrderBook(self, symbol):  # FIXME
        url = self.apiurl + '/api/v3/market/orderbook/level2'
        headers = authentication(method='get', endpoint='/api/v3/market/orderbook/level2')
        parameters = {"symbol": symbol}
        r = self.session.get(url, headers=headers, params=parameters)
        data = r.json()
        return data
    
    def placeOrder(self, details): # FIXME
        detailStr = json.dumps(details)
        url = self.apiurl + '/api/v1/orders'
        headers = authentication('post', '/api/v1/orders')
        return self.session.post(url, headers=headers, data=detailStr, timeout=5).json()
    
    def cancelOrder(self, orderId): # FIXME
        detailStr = json.dumps(details)
        url = self.apiurl + '/api/v1/orders/' + orderId
        headers = authentication('delete', '/api/v1/orders/'+ orderId)
        return self.session.post(url, headers=headers, data=detailStr, timeout=5).json()
    
    def cancelAllOrders(self):
        pass


In [30]:
details = {"clientOid": str(1), "side": "buy", "symbol": "ETH-USDT", "type": "limit", "price": "1", "size": "0.1"}
json.dumps(details)

'{"clientOid": "1", "side": "buy", "symbol": "ETH-USDT", "type": "limit", "price": "1", "size": "0.1"}'

In [31]:
kucoinAPI = PrivateData()

In [32]:
kucoinAPI.getBalance()

KeyError: 'data'

In [33]:
kucoinAPI.getFullOrderBook('ETH-USDT')

{'code': '400005', 'msg': 'Invalid KC-API-SIGN'}

In [34]:
kucoinAPI.placeOrder(details)

{'code': '400005', 'msg': 'Invalid KC-API-SIGN'}